# Loading CAO points information into a pandas data frame, using CAO website

http://www.cao.ie/index.php?page=points&p=2021

***

## INTRODUCTION

## WHAT IS CAO

### LOADING THE DATA

In [1]:
# HTTP request
import requests as rq
# Regular expressions
import re
# Dates and time
import datetime as dt
# Data frames
import pandas as pd
# For downloading
import urllib.request as urlrq
# PDF
import camelot

## LEVEL 8 POINTS 2019, 2020, 2021
### ROUND 1 AND ROUND 2

In [2]:
# Current date and time
now = dt.datetime.now()
# Format as a string
nowstr = (now.strftime("%Y%m%d_%H%M%S"))

In [3]:
# Regular expression
re_course = re.compile(r"([A-Z]{2}[0-9]{3})(.*)")

In [4]:
# Defining a dictionary
    # Using website archive to access the wanted data (2019, 2020)
years_dict = {
    "2019": [("data/cao2019_" +  nowstr), "https://web.archive.org/web/20191019135815/http://www2.cao.ie:80/points/l8.php"],
    "2020": [("data/cao2020_" +  nowstr), "https://web.archive.org/web/20201108133105/http://www2.cao.ie/points/l8.php"],
    "2021": [("data/cao2021_" +  nowstr), "http://www2.cao.ie/points/l8.php"]
}

In [5]:
# Loop through the (dict) years
for year, content in years_dict.items():
    # Fetch the CAO points URL
    rq.get (content[1])
    resp = rq.get (content[1])
    # The server uses the wrong encoding
        # Change to "cp1252"
    resp.encoding = "cp1252"
    # Check if OK:
        #Response [200] means OK
    print (resp)
    # Save the original html file
    with open(content[0] + ".html", "w") as f:
        f.write(resp.text)
    # Keep track of how many courses we process
    no_lines = 0
    # Iterating through the lines
    resp.iter_lines()
    resps = resp.iter_lines()
    # Open the csv file for writing.
    with open(content[0] + ".csv", "w") as f:
        # Write a header row.
        f.write(','.join(["CODE", "TITLE", "R1_POINTS", "R2_POINTS"]) + "\n")
        # Loop through lines of the response.
        for line in resps:
            # Decode the line, using the wrong encoding
            dline = line.decode("cp1252")
            # Match only the lines representing courses
            if re_course.fullmatch(dline):
                # Add one to the lines counter
                no_lines = no_lines + 1
                # The course code
                course_code = dline[:5]
                # The course title
                course_title = dline[7:57]
                # Round one points
                course_points = re.split(' +', dline[60:])
                if len(course_points) != 2:
                    course_points = course_points[:2]
                # Join the fields using a comma
                linesplit = [course_code, course_title, course_points[0], course_points[1]]
                # Rejoin the substrings with commas in between
                f.write(",".join(linesplit) + "\n")   
    # Print the total number of processed lines
    print(f"Total number of lines is {no_lines}.")

<Response [200]>
Total number of lines is 930.
<Response [200]>
Total number of lines is 961.
<Response [200]>
Total number of lines is 949.


## LEVEL 8 POINTS 2020, 2019
### EOS AND MED
http://www.cao.ie/index.php?page=points&p=2020

### 2020 .xlsx

In [6]:
# Create a file path for the original data
path = ("data/cao2020_eos" +  nowstr + ".xlsx")

In [7]:
urlrq.urlretrieve('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx', path)

('data/cao2020_eos20211115_194046.xlsx',
 <http.client.HTTPMessage at 0x1ecd369ab80>)

In [8]:
# Download and parse the excel spreadsheet
df = pd.read_excel("http://www2.cao.ie/points/CAOPointsCharts2020.xlsx", skiprows = 10)
#df

In [9]:
# Spotcheck random row
#df.iloc[753]

In [10]:
# Spotcheck last row
#df.iloc[-1]

In [11]:
# Create a file path for the pandas data
path = ("data/cao2020_eos" +  nowstr + ".csv")

In [12]:
# Save pandas data frame to disk
df.to_csv(path)

***

### 2019 .pdf

### GET THE ORIGINAL

In [13]:
path = 'data/cao2019_eos' + nowstr + '.pdf'

In [14]:
resp_pdf = rq.get("http://www2.cao.ie/points/lvl8_19.pdf")
resp_pdf

<Response [200]>

In [15]:
with open(path, 'wb') as f:
    f.write(resp_pdf.content)

### READ THE TABLE FROM PDF

In [16]:
file = path
tables = camelot.read_pdf(file, pages = "all", flavor = "lattice")

In [17]:
print ("Tables:", tables.n)

Tables: 18


https://stackoverflow.com/questions/52383287/concatenate-dataframes-in-a-for-loop

In [18]:
table_total = []
for x in range (0,18):
    df = tables[x].df
    table_total.append(df)
    
table = pd.concat(table_total)
table.columns = ["CODE","TITLE", "EOS_2019", "MID_2019"]
table.sort_values("CODE", inplace = True)
table = table.iloc[35: , :]
table.to_csv("data/cao2019_EOS" +  nowstr + ".csv", index = False)
table

,CODE,TITLE,EOS_2019,MID_2019
21,AC120,International Business,234,269
22,AC137,Liberal Arts,252,275
3,AD101,First Year Art & Design (Common Entry),# +matric 550,
4,AD102,Graphic Design and Moving Image Design,# +matric 635,
5,AD103,Textile & Surface Design and Jewellery & Objects,# +matric 545,
...,...,...,...,...
37,WD200,Arts (options),221,296
38,WD210,Software Systems Development,271,329
39,WD211,Creative Computing,275,322
40,WD212,Recreation and Sport Management,274,311


# READ AND JOIN ALL TABLES - TESTING

In [19]:
df2019 = pd.read_csv(("data/cao2019_" +  nowstr + ".csv"), encoding='cp1252')
#df2019

In [20]:
df2019_eos = pd.read_csv ("data/cao2019_EOS" +  nowstr + ".csv")
#df2019_eos

In [21]:
df2020 = pd.read_csv(("data/cao2020_" +  nowstr + ".csv"), encoding='cp1252')
#df2020

In [22]:
df2021 = pd.read_csv(("data/cao2021_" +  nowstr + ".csv"), encoding='cp1252')
#df2021

In [23]:
courses2019 = df2019[["CODE", "TITLE"]]
#courses2019

In [24]:
courses2020 = df2020[["CODE", "TITLE"]]
#courses2020

In [25]:
courses2021 = df2021[["CODE", "TITLE"]]
#courses2021

In [26]:
allcourses = pd.concat([courses2019, courses2020, courses2021])
#allcourses

In [27]:
allcourses.drop_duplicates(subset=["CODE"], inplace=True, ignore_index=True)

In [28]:
#allcourses

In [29]:
df2019.columns = ["CODE","TITLE", "R1_POINTS_2019", "R2_POINTS_2019"]
df2019.set_index("CODE", inplace=True)
#df2019

In [30]:
df2019_eos.set_index("CODE", inplace=True)

In [31]:
df2020.columns = ["CODE","TITLE", "R1_POINTS_2020", "R2_POINTS_2020"]
df2020.set_index("CODE", inplace=True)
#df2020

In [32]:
df2021.columns = ["CODE","TITLE", "R1_POINTS_2021", "R2_POINTS_2021"]
df2021.set_index("CODE", inplace=True)
#df2021

In [33]:
allcourses.set_index("CODE", inplace=True)
allcourses = allcourses.join(df2019[["R1_POINTS_2019", "R2_POINTS_2019"]])
#allcourses

In [34]:
allcourses = allcourses.join(df2019_eos[["EOS_2019", "MID_2019"]])
#allcourses

In [35]:
allcourses = allcourses.join(df2020[["R1_POINTS_2020", "R2_POINTS_2020"]])
#allcourses

In [36]:
allcourses = allcourses.join(df2021[["R1_POINTS_2021", "R2_POINTS_2021"]])
#allcourses

In [37]:
allcourses.sort_values("CODE", inplace = True)
allcourses.to_csv ("Rounds.csv")
allcourses

,TITLE,R1_POINTS_2019,R2_POINTS_2019,EOS_2019,MID_2019,R1_POINTS_2020,R2_POINTS_2020,R1_POINTS_2021,R2_POINTS_2021
CODE,,,,,,,,,
AC120,International Business ...,234,234,234,269,209,NaN,294,294
AC137,Liberal Arts ...,275,252,252,275,252,NaN,271,270
AD101,First Year Art &amp; Design (Common Entry) ...,NaN,#+Matric,# +matric 550,NaN,NaN,#+matric,#554,NaN
AD102,Graphic Design and Moving Image Design ...,#+Matric,NaN,# +matric 635,NaN,#+matric,NaN,#538,NaN
AD103,Textile &amp; Surface Design and Jewellery &am...,cts,#+Matric,# +matric 545,NaN,cts,(,#505,NaN
...,...,...,...,...,...,...,...,...,...
WD211,Creative Computing ...,275,NaN,275,322,271,NaN,270,NaN
WD212,Recreation and Sport Management ...,274,NaN,274,311,270,NaN,262,NaN
WD230,Mechanical and Manufacturing Engineering ...,273,NaN,273,348,253,NaN,230,230


# END TEST

### DATA COMPARISON

# CONCLUSION

***

## REFERENCES

## LINKS TO CHECK